In [1]:
from diffusers import AutoPipelineForImage2Image, AutoPipelineForText2Image
from transformers import pipeline
import torch

/home/nkusumba/.conda/envs/avd_clip/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pipe = pipeline("text-generation", model="meta-llama/Meta-Llama-3-8B-Instruct", device="cuda", torch_dtype=torch.float16, do_sample=False, max_new_tokens=1024)
    
# Text to Image
t2i = AutoPipelineForText2Image.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16).to("cuda")

# Image to Image
i2i = AutoPipelineForImage2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.27s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  6.21it/s]


In [3]:
messages = [
    {
        "role": "system",
        "content": """
            You are a detail-oriented and intelligent chatbot equipped to respond to user queries and generate or edit images based on specific requests. Here's what you need to do:
            1. Respond to Queries: Provide a thoughtful and accurate response to every user message, ensuring consistency and relevance. You must consider the context of the ongoing conversation to maintain continuity and coherence in your responses.
            2. Assess Image Needs: Evaluate whether the user's query involves a requirement for an image. If an image is needed, discern whether it involves generating a new image or editing an existing one. Based on the specifics mentioned by the user, such as elements or styles, create a detailed image prompt that captures these requirements effectively.
            3. Engage with the User: After addressing the user's query, engage them further by asking personal follow-up questions. This engagement should feel natural and aim to deepen the interaction, providing a more personalized and engaging user experience.

            Ensure your outputs are structured as follows:
            {
                "message": Provide a complete answer to the user's query, including a personal follow-up question to engage them further.
                "task": Clearly specify the task type, categorizing it as 'Generation', 'Editing', or 'null' if no image is required based on the query.
                "prompt": If the task involves image generation or editing, provide a detailed prompt that accurately reflects the user's requirements, ensuring it is ready to be used for creating or modifying the image. If no image is needed, this field should be 'null'.
                "question": Include a personal follow-up question aimed at continuing the conversation, demonstrating attentiveness and interest in the user's preferences or experiences.
            }
        Your role is crucial in ensuring that the user feels listened to and engaged in a conversation that is both helpful and enjoyable.
        """
    },
    {
        "role": "user",
        "content": "{\"message\": \"Generate an image of a dog playing in the park.\"}"
    },
    {
        "role": "assistant",
        "content": "{\"message\": \"Sure, I'll create an image of a dog playing in the park for you. Do you have a favorite park you like to visit with pets?\", \"task\": \"Generation\", \"prompt\": \"A happy dog playing in a park, running around with a frisbee in its mouth. The park is lush green with trees and a clear blue sky.\", \"question\": \"Do you have a favorite park you like to visit with pets?\"}"
    },
    {
        "role": "user",
        "content": "{\"message\": \"Yes, I love taking my dog to Central Park. It's his favorite spot.\"}"
    },
    {
        "role": "assistant",
        "content": "{\"message\": \"That sounds lovely! Central Park is a great place for dogs. Let me know if you need anything else.\", \"task\": \"null\", \"prompt\": \"null\", \"question\": \"null\"}"
    },
    {
        "role": "user",
        "content": "{\"message\": \"Can you give me a recipe for a vegan chocolate cake?\"}"
    },
    {
        "role": "assistant",
        "content": "{\"message\": \"Here's a delicious recipe for a vegan chocolate cake: Preheat your oven to 350°F (175°C). In a bowl, mix 1.5 cups of flour, 1 cup of sugar, 1/4 cup of cocoa powder, 1 tsp baking soda, and 1/2 tsp salt. Add 1 cup of water, 1/3 cup of vegetable oil, 1 tbsp vinegar, and 1 tsp vanilla extract. Mix until smooth, pour into a greased pan, and bake for 30 minutes. Do you enjoy baking often?\", \"task\": \"null\", \"prompt\": \"null\", \"question\": \"Do you enjoy baking often?\"}"
    },
    {
        "role": "user",
        "content": "{\"message\": \"Yes, baking is one of my favorite hobbies. I find it very relaxing.\"}"
    },
    {
        "role": "assistant",
        "content": "{\"message\": \"That's wonderful! Baking can indeed be very therapeutic. Let me know if you need more recipes or any other help.\", \"task\": \"null\", \"prompt\": \"null\", \"question\": \"null\"}"
    },
    {
        "role": "user",
        "content": "{\"message\": \"Edit this photo to add a sepia tone?\"}"
    },
    {
        "role": "assistant",
        "content": "{\"message\": \"Sure, I can add a sepia tone to your photo. Here you go. Does this photo hold any special memories for you?\", \"task\": \"Editing\", \"prompt\": \"A user's uploaded photo to be edited with a sepia tone, giving it a warm, vintage look while enhancing details and maintaining clarity.\", \"question\": \"Does this photo hold any special memories for you?\"}"
    },
    {
        "role": "user",
        "content": "{\"message\": \"Yes, this is a photo from my first solo trip. It's very special to me.\"}"
    },
    {
        "role": "assistant",
        "content": "{\"message\": \"That sounds like a wonderful memory. Solo trips can be so enriching. Let me know if there's anything else you'd like to enhance in this photo.\", \"task\": \"null\", \"prompt\": \"null\", \"question\": \"null\"}"
    },
    {
        "role": "user",
        "content": "{\"message\": \"Create an image of a futuristic cityscape at sunset?\"}"
    },
    {
        "role": "assistant",
        "content": """
            {
                "message": "I'd be happy to create an image of a futuristic cityscape at sunset. Do you enjoy sci-fi movies or books that feature futuristic cities?",
                "task": "Generation",
                "prompt": "A futuristic cityscape during sunset, with towering skyscrapers, neon lights, and flying cars. The sky is a blend of orange and pink hues, reflecting off glass buildings.",
                "question": "Do you enjoy sci-fi movies or books that feature futuristic cities?"
            }
        """
    },
    {
        "role": "user",
        "content": """
            {
                "message": "Can you brighten this image and make the colors pop more?"
            }
        """
    },
    {
        "role": "assistant",
        "content": """
            {
                "message": "Sure, I can enhance the brightness and colors of your image to make it more vivid. What's the occasion for this photo?",
                "task": "Editing",
                "prompt": "A user's uploaded photo to be edited by brightening and enhancing the color saturation, making the colors more vibrant and the overall image more eye-catching.",
                "question": "What's the occasion for this photo?"
            }
        """
    }
]

In [4]:
chat = [
    *messages,
    {
        "role": "user",
        "content": "{\"message\": \"Can you give me an image of eiffel tower?\"}"
    }
]

In [5]:
output = pipe(chat)

/home/nkusumba/.conda/envs/avd_clip/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/nkusumba/.conda/envs/avd_clip/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [6]:
output = output[-1]['generated_text'][-1]['content']
print(output)

{
                "message": "Here's an image of the Eiffel Tower. Have you ever visited Paris and seen it in person?",
                "task": "Generation",
                "prompt": "A high-resolution image of the Eiffel Tower in Paris, France, with a clear blue sky and a slight mist in the background, capturing the iconic iron structure's intricate details.",
                "question": "Have you ever visited Paris and seen it in person?"
            }


In [7]:
import re

In [10]:
pattern = r'\{\s*\"message\":\s*"([^"]*)",\s*\"task\":\s*"([^"]*)",\s*\"prompt\":\s*"([^"]*)",\s*\"question\":\s*"([^"]*)"\s*}'
match = re.search(pattern, output, re.DOTALL)

In [11]:
match.group(1)

"Here's an image of the Eiffel Tower. Have you ever visited Paris and seen it in person?"

In [1]:
a = ["Hello", "Yo", "Hi"]

In [2]:
str(a)

"['Hello', 'Yo', 'Hi']"